In [1]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
import time
from transformers import AutoModel
from transformers import AutoConfig

torch.set_grad_enabled(False)

In [13]:
from calibrate_e_ml import get_module_info
from cg.node import construct_aggregation_graph

In [18]:
seq_len = 100
bs = 4
input_ids = torch.randint(1000, size=(bs, seq_len), dtype=torch.long, device='cpu')
model_name = "distilbert-base-uncased"
cuda_exist = torch.cuda.is_available()
device = torch.device("cuda" if cuda_exist else "cpu")
information = get_module_info(model_name, bs, seq_len, device, 'module')

In [20]:
information.keys()

dict_keys(['BertSelfAttention', 'BertSelfOutput', 'BertAttention', 'BertIntermediate', 'BertOutput', 'BertLayer', 'BertEncoder', 'BertPooler', 'Embeddings', 'FFN'])

In [21]:
config = AutoConfig.from_pretrained(model_name)
config.torchscript = True
model = AutoModel.from_config(config).eval()

In [22]:
# model

In [23]:
from torch import nn
def is_ignore_operation(module):
    ignore_operations = {nn.Dropout, }
    for e in ignore_operations:
        if isinstance(module, e):
            return True
    return False

In [24]:
start_times = dict()
end_times = dict()
module_inputs = dict()
module_outputs = dict()
modules = dict()
m_ins = {}
m_outs = {}

def log_end_builder(name):
    def log_end(module, m_in, m_out):
        k = f'{name}:{module.__class__.__name__}'
        end_times[k] = time.perf_counter()
#         print(k)
        m_ins[k]=m_in
        m_outs[k]= m_out
#         module_inputs[f'{name}:{module.__class__.__name__}'] = (
#             m_in[0].shape, m_in[0].dtype)
#         module_outputs[f'{name}:{module.__class__.__name__}'] = (
#             m_out.shape, m_out.dtype)
        modules[f'{name}:{module.__class__.__name__}'] = module

    return log_end

In [25]:
for (name, module) in model.named_modules():
    if not name:
        continue
    # module.register_forward_pre_hook(log_start_builder(name))
    if is_ignore_operation(module):
        continue
    module.register_forward_hook(log_end_builder(name))
inputs = torch.randint(1000, size=(4, 100)).long()
_ = model(input_ids=inputs, return_dict=True)

In [28]:
for k, m_in in m_ins.items():
    print(k, [t.shape if isinstance(t, torch.Tensor) else t for t in m_in] if m_in else m_in)
    if m_in:
        mo=modules[k](*m_in)
        print(type(m_in), len(m_in), len(mo))

embeddings.word_embeddings:Embedding [torch.Size([4, 100])]
<class 'tuple'> 1 4
embeddings.position_embeddings:Embedding [torch.Size([4, 100])]
<class 'tuple'> 1 4
embeddings.LayerNorm:LayerNorm [torch.Size([4, 100, 768])]
<class 'tuple'> 1 4
embeddings:Embeddings [torch.Size([4, 100])]
<class 'tuple'> 1 4
transformer.layer.0.attention.q_lin:Linear [torch.Size([4, 100, 768])]
<class 'tuple'> 1 4
transformer.layer.0.attention.k_lin:Linear [torch.Size([4, 100, 768])]
<class 'tuple'> 1 4
transformer.layer.0.attention.v_lin:Linear [torch.Size([4, 100, 768])]
<class 'tuple'> 1 4
transformer.layer.0.attention.out_lin:Linear [torch.Size([4, 100, 768])]
<class 'tuple'> 1 4
transformer.layer.0.attention:MultiHeadSelfAttention ()
transformer.layer.0.sa_layer_norm:LayerNorm [torch.Size([4, 100, 768])]
<class 'tuple'> 1 4
transformer.layer.0.ffn.lin1:Linear [torch.Size([4, 100, 768])]
<class 'tuple'> 1 4
transformer.layer.0.ffn.lin2:Linear [torch.Size([4, 100, 3072])]
<class 'tuple'> 1 4
transform

In [24]:
k='encoder.layer.1.attention.output:BertSelfOutput'


encoder.layer.1.attention.output.dense:Linear
encoder.layer.1.attention.output.dense:Linear
encoder.layer.1.attention.output.LayerNorm:LayerNorm
encoder.layer.1.attention.output.LayerNorm:LayerNorm
encoder.layer.1.attention.output:BertSelfOutput
encoder.layer.1.attention.output:BertSelfOutput


tensor([[[-0.0211,  1.0562,  1.8278,  ...,  1.2891, -0.3977,  0.7767],
         [ 1.8768,  0.3936,  0.6516,  ...,  1.3038,  1.5040,  0.5726],
         [ 0.4111,  0.5431,  0.0584,  ...,  0.0996,  0.6414,  1.0507],
         ...,
         [ 1.4097,  0.8684,  0.5895,  ...,  1.4621,  1.3316,  0.6746],
         [ 2.1392,  1.1131,  0.6786,  ...,  1.6456, -0.8073,  2.6841],
         [-0.4328,  1.0356, -0.3433,  ...,  1.0591,  1.4426, -0.4560]],

        [[ 0.1866,  0.3579,  2.3137,  ...,  0.9798, -0.3501,  0.6097],
         [ 2.1507, -0.2397,  0.1715,  ...,  0.9048,  2.0328, -0.7634],
         [ 2.0196, -0.1552,  0.9312,  ...,  0.4331, -0.5626,  1.0746],
         ...,
         [ 1.2808,  0.7583, -0.1189,  ...,  0.1574,  0.7162, -0.5988],
         [ 1.2116,  0.8162, -0.2515,  ...,  0.8901, -0.4312,  0.2813],
         [-0.1550,  1.7520,  0.1550,  ...,  1.2414,  2.1268,  0.9838]],

        [[ 0.5811,  2.1213,  1.2449,  ...,  1.0366,  0.5368,  0.3451],
         [ 1.6340, -0.6104, -0.6088,  ...,  0

In [23]:
m_ins[k]

(tensor([[[ 0.0642,  0.2725,  0.0914,  ..., -0.2159, -0.0125,  0.0897],
          [ 0.0625,  0.2715,  0.0923,  ..., -0.2154, -0.0096,  0.0909],
          [ 0.0633,  0.2717,  0.0921,  ..., -0.2160, -0.0088,  0.0898],
          ...,
          [ 0.0651,  0.2715,  0.0912,  ..., -0.2163, -0.0111,  0.0906],
          [ 0.0640,  0.2716,  0.0919,  ..., -0.2157, -0.0107,  0.0890],
          [ 0.0627,  0.2711,  0.0913,  ..., -0.2165, -0.0094,  0.0909]],
 
         [[ 0.0757,  0.2901,  0.1288,  ..., -0.1862, -0.0213,  0.0939],
          [ 0.0748,  0.2919,  0.1301,  ..., -0.1863, -0.0190,  0.0940],
          [ 0.0743,  0.2909,  0.1285,  ..., -0.1860, -0.0195,  0.0949],
          ...,
          [ 0.0761,  0.2903,  0.1306,  ..., -0.1872, -0.0215,  0.0940],
          [ 0.0756,  0.2911,  0.1293,  ..., -0.1852, -0.0197,  0.0949],
          [ 0.0739,  0.2921,  0.1310,  ..., -0.1864, -0.0187,  0.0942]],
 
         [[ 0.0488,  0.2816,  0.1362,  ..., -0.2097,  0.0043,  0.1039],
          [ 0.0487,  0.2832,

In [312]:
for name, module in model.named_modules():
    print(name, module.__class__.__name__)

 BertModel
embeddings BertEmbeddings
embeddings.word_embeddings Embedding
embeddings.position_embeddings Embedding
embeddings.token_type_embeddings Embedding
embeddings.LayerNorm LayerNorm
embeddings.dropout Dropout
encoder BertEncoder
encoder.layer ModuleList
encoder.layer.0 BertLayer
encoder.layer.0.attention BertAttention
encoder.layer.0.attention.self BertSelfAttention
encoder.layer.0.attention.self.query Linear
encoder.layer.0.attention.self.key Linear
encoder.layer.0.attention.self.value Linear
encoder.layer.0.attention.self.dropout Dropout
encoder.layer.0.attention.output BertSelfOutput
encoder.layer.0.attention.output.dense Linear
encoder.layer.0.attention.output.LayerNorm LayerNorm
encoder.layer.0.attention.output.dropout Dropout
encoder.layer.0.intermediate BertIntermediate
encoder.layer.0.intermediate.dense Linear
encoder.layer.0.output BertOutput
encoder.layer.0.output.dense Linear
encoder.layer.0.output.LayerNorm LayerNorm
encoder.layer.0.output.dropout Dropout
encoder.lay

In [25]:
level = information['linear'][0]
print(level)
fn = level['module']
fname=level['name']
fi_size, fi_dtype = level['inputs']
fo_size, fo_dtype = level['outputs']
fi = torch.rand(fi_size, dtype=fi_dtype)
fo = torch.rand(fo_size, dtype=fo_dtype)

fo_call = fn(fi)

{'name': 'encoder.layer.0.attention.self.query:Linear', 'module': Linear(in_features=128, out_features=128, bias=True), 'inputs': (torch.Size([4, 100, 128]), torch.float32), 'outputs': (torch.Size([4, 100, 128]), torch.float32), 'runtime': 0.012598949000000914}


In [26]:
f_start = time.perf_counter()
for _ in range(10000):
    fo_call = fn(fi)
f_end = time.perf_counter()
print(f_end-f_start)

4.047626644999809


In [27]:
trace = torch.jit.trace(fn, fi)
trace_graph = trace.inlined_graph
trace_graph

graph(%self : __torch__.torch.nn.modules.linear.___torch_mangle_1.Linear,
      %input : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu)):
  %2 : Tensor = prim::GetAttr[name="bias"](%self)
  %3 : Tensor = prim::GetAttr[name="weight"](%self)
  %4 : Float(128:1, 128:128, requires_grad=1, device=cpu) = aten::t(%3) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %output : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::matmul(%input, %4) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %6 : int = prim::Constant[value=1]() # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  %m_out : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::add_(%output, %2, %6) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  return (%m_out)

In [28]:
graph, op_data_types = construct_aggregation_graph(trace_graph, fname)

/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for prim::GetAttr!
  else:
/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for aten::t!
  else:
/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for prim::Constant!
  else:


In [29]:
graph

Graph(name=encoder.layer.0.attention.self.query:Linear,
	inputs=[
		DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)
		DataNode(id=input, dtype=Float, shape=[4, 100, 128])],
	nodes=[
		OpNode(id=2, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)],
			outputs=[
				DataNode(id=2, dtype=None)],)
		OpNode(id=3, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)],
			outputs=[
				DataNode(id=3, dtype=None)],)
		OpNode(id=4, op=aten::t, 
			inputs=[
				DataNode(id=3, dtype=None)],
			outputs=[
				DataNode(id=4, dtype=Float, shape=[128, 128])],)
		OpNode(id=output, op=aten::matmul, flops=13107200, mem_bytes=671744, 
			inputs=[
				DataNode(id=input, dtype=Float, shape=[4, 100, 128])
				DataNode(id=4, dtype=Float, shape=[128, 128])],
			outputs=[
				DataNode(id=output, dtype=Float, shape=[4, 100, 128])],)
		OpNode(i

```
for each input size (bs, seq_len) pair:
    for each module:
        get jit trace, extract flops and mem_bytes,
        log (bs, seq_len, name, module level, flops, mem_bytes)
        run the module 100k times, log timestamp
        sleep 5s
        repeat 10 rounds
record energy with timestamp, log cpu_util, gpu_util, mem_util, gpu_util

extract data (bs, seq_len, name, module level, flops, mem_bytes, cpu_util, gpu_util, mem_util, gpu_util, energy)
calc energy variance, simple linear regression fitting
```   

In [30]:
from collections import defaultdict

def get_flops_mem_bytes(graph):
    flops = dict()
    mem_bytes = dict()

    for node in graph.nodes:
        flops[node.id] = node.flops
        mem_bytes[node.id] = node.mem_bytes
        if node.flops:
            print(node.op, node.flops, node.mem_bytes)
    return sum(flops.values()), sum(mem_bytes.values())

In [31]:
get_flops_mem_bytes(graph)

aten::matmul 13107200 671744
aten::add_ 51200 409600


(13158400, 1081344)

In [ ]:
from datetime import datetime
print(datetime.fromtimestamp(1605498402.136237))

In [118]:
import pandas
pandas.set_option('display.float_format', '{:.6f}'.format)

res= pandas.read_csv('data/mlexp/mlexp-linear-res2.csv')
energy= pandas.read_csv('data/mlexp/mlexp-linear-energy2.csv', error_bad_lines=False, usecols=[0, 2])

In [88]:
import json

model_name = 'bert-base-uncased'
level = 'linear'
runs = 10
bs = 16
seq_len = 384

prof_filename = f'{model_name}_{level}_r{runs}_b{bs}_i{seq_len}.json'

with open(f'data/mlexp/{prof_filename}') as f:
    prof_info = json.load(f)

In [89]:
prof_info[0]

{'name': 'encoder.layer.0.attention.self.query:Linear',
 'flops': 7252475904,
 'mem_bytes': 113246208,
 'start_1': 1610903590.6697361,
 'end_1': 1610903591.3599648,
 'start_2': 1610903592.3611808,
 'end_2': 1610903593.052057,
 'start_3': 1610903594.0532901,
 'end_3': 1610903594.7442207,
 'start_4': 1610903595.7448068,
 'end_4': 1610903596.435857,
 'start_5': 1610903597.4370837,
 'end_5': 1610903598.1282494,
 'start_6': 1610903599.1294367,
 'end_6': 1610903599.824273,
 'start_7': 1610903600.8254433,
 'end_7': 1610903601.5203035,
 'start_8': 1610903602.5215309,
 'end_8': 1610903603.216559,
 'start_9': 1610903604.2176173,
 'end_9': 1610903604.9122813,
 'start_10': 1610903605.913508,
 'end_10': 1610903606.6082392,
 'type': 'linear'}

In [119]:
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

energy=energy[energy['timestamp'].apply(lambda x: is_float(x))]
energy['value']=energy['value'].div(100)
energy['timestamp']=energy['timestamp'].astype(float)

In [129]:
res_np = res.to_numpy()
energy_np = energy.to_numpy()

In [131]:
res

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
0,1610903583.440226,2.600000,4.000000,2,0,0,15313,2,0,34326
1,1610903583.540479,1.800000,4.000000,2,0,0,15313,0,0,34343
2,1610903583.640747,0.100000,4.000000,2,0,0,15313,0,0,34491
3,1610903583.741015,0.100000,4.000000,2,0,0,15216,0,0,34587
4,1610903583.841277,0.100000,4.000000,2,0,0,15313,0,0,34491
...,...,...,...,...,...,...,...,...,...,...
1684,1610903752.278331,0.100000,4.000000,2,0,0,10150,0,0,6495
1685,1610903752.378598,0.100000,4.000000,2,0,0,10059,0,0,6400
1686,1610903752.478866,0.100000,4.000000,2,0,0,9861,0,0,6592
1687,1610903752.579133,0.100000,4.000000,2,0,0,10052,0,0,6397


In [132]:
res_t = res_np[:, 0]
energy_t = energy_np[:, 0]

In [133]:
energy_np

array([[1.61090358e+09, 1.12330000e+02],
       [1.61090358e+09,            nan],
       [1.61090358e+09, 1.00000000e-02],
       ...,
       [1.61090376e+09, 1.15410000e+02],
       [1.61090376e+09, 1.15140000e+02],
       [1.61090376e+09, 1.13470000e+02]])

In [134]:
import bisect

res_s = bisect.bisect_right(res_t, prof_info[0]['start_1'])
res_e = bisect.bisect_right(res_t, prof_info[0]['end_1'])

energy_s = bisect.bisect_right(energy_t, prof_info[0]['start_1'])
energy_e = bisect.bisect_right(energy_t, prof_info[0]['end_1'])

In [138]:
res[res_s:res_e]

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
73,1610903590.760744,9.300000,9.000000,2,99,50,208903,0,0,34360
74,1610903590.861005,8.900000,9.000000,2,99,50,212446,0,0,34502
75,1610903590.961265,10.400000,9.000000,2,100,39,210800,0,0,34502
76,1610903591.061532,9.200000,9.000000,2,100,39,212381,0,0,34511
77,1610903591.161789,8.400000,9.000000,2,100,39,211546,0,0,34502
78,1610903591.262048,8.400000,9.000000,2,100,39,211469,0,0,34502


In [215]:
res

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
0,1610914901.591612,1.100000,4.000000,2,0,0,15846,0,0,34087
1,1610914901.761873,0.600000,4.000000,2,0,0,15751,0,0,34135
2,1610914901.932141,0.100000,4.000000,2,0,0,15749,0,0,34149
3,1610914902.102404,0.100000,4.000000,2,0,0,15312,0,0,34149
4,1610914902.272667,0.100000,4.000000,2,0,0,15654,0,0,34246
...,...,...,...,...,...,...,...,...,...,...
64732,1610926947.577797,8.200000,9.000000,2,100,50,211232,0,0,6603
64733,1610926947.748063,8.100000,9.000000,2,100,51,213543,0,0,6603
64734,1610926947.918305,8.200000,9.000000,2,100,50,210710,0,0,6603
64735,1610926948.088569,8.100000,9.000000,2,100,50,214403,0,0,6698


In [147]:
import numpy as np

for prof_item in prof_info:
    energy_runs = []
    for r in range(1, runs+1):
        energy_s = bisect.bisect_right(energy_t, prof_item[f'start_{r}'])
        energy_e = bisect.bisect_right(energy_t, prof_item[f'end_{r}'])
        energy_r = energy[energy_s:energy_e]['value'].sum()
        energy_runs.append(energy_r)
    std_percent = np.std(energy_runs) / np.mean(energy_runs)
    mean = np.mean(energy_runs)
    print(prof_item['type'], prof_item['name'], std_percent, mean)

linear encoder.layer.0.attention.self.query:Linear 0.07236582207171405 1242.308
linear encoder.layer.0.intermediate.dense:Linear 0.13399399857273184 8558.122
linear encoder.layer.0.output.dense:Linear 0.12190375616305438 7355.465000000001
linear pooler.dense:Linear 0.8961761566460925 93.628


In [257]:
import bisect
import json
import pandas as pd
import numpy as np
from pathlib import Path
from common import sanitize

pd.set_option('display.float_format', '{:.6f}'.format)
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [301]:
res_names = ['cpu', 'mem', 'gpu', 'gpu_mem']

feature_names = ['batch_size', 'seq_len', 'flops',
                 'mem_bytes'] + \
                ['times_mean', 'times_std',
                 'gpu_power_mean', 'gpu_power_std',
                 'energy_mean', 'energy_std',
                 'level_name', 'model_name'] + res_names + [f'{k}_std' for k in res_names] 

def process_record(energy, prof_info, res, feature_values,
                   model_name, bs, runs, seq_len):
    res_np = res.to_numpy()
    res_t = res_np[:, 0]
    energy_np = energy.to_numpy()
    energy_t = energy_np[:, 0]
    for prof_item in prof_info:
        gpu_power_runs = []
        energy_runs = []
        times_runs = []
        res_runs = {k: [] for k in res_names}
        repeats = prof_item['repeats']
        for r in range(1, runs + 1):
            start_r = prof_item[f'start_{r}']
            end_r = prof_item[f'end_{r}']
            times_runs.append(end_r - start_r)

            res_s = bisect.bisect_right(res_t, start_r)
            res_e = bisect.bisect_right(res_t, end_r)
            res_r = res[res_s:res_e]
            for rn in res_names:
                res_runs[rn].append(res_r[rn].mean())
            gpu_power_r = res_r['gpu_power'].sum()
            gpu_power_runs.append(gpu_power_r)

            e_s = bisect.bisect_right(energy_t, start_r)
            e_e = bisect.bisect_right(energy_t, end_r)
            energy_r = energy[e_s:e_e]['value'].div(repeats).sum()
            energy_runs.append(energy_r)

        times_mean = np.mean(times_runs)
        times_std = np.std(times_runs) / times_mean * 100
        gpu_power_mean = np.mean(gpu_power_runs)
        gpu_power_std = np.std(gpu_power_runs) / gpu_power_mean * 100
        energy_mean = np.mean(energy_runs)
        energy_std = np.std(energy_runs) / energy_mean * 100
        for rn in res_names:
            feature_values[rn].append(np.mean(res_runs[rn]))
            rn_std = np.std(res_runs[rn]) / np.mean(res_runs[rn]) * 100
            feature_values[f'{rn}_std'].append(rn_std)

        flops = prof_item['flops'] / 1e6
        mem_bytes = prof_item['mem_bytes'] / 1024 / 1024
        feature_values['batch_size'].append(bs)
        feature_values['seq_len'].append(seq_len)
        feature_values['energy_mean'].append(energy_mean)
        feature_values['energy_std'].append(energy_std)
        feature_values['gpu_power_mean'].append(gpu_power_mean)
        feature_values['gpu_power_std'].append(gpu_power_std)
        feature_values['flops'].append(flops)
        feature_values['mem_bytes'].append(mem_bytes)
        feature_values['times_mean'].append(times_mean)
        feature_values['times_std'].append(times_std)
        feature_values['level_name'].append(prof_item['name'])
        feature_values['model_name'].append(model_name)


In [302]:
args=Namespace(out_dir='data/small-exp', 
               models=["google/mobilebert-uncased", "bert-base-uncased", "distilbert-base-uncased", "roberta-base"],
               exp_type='ml', exp_name='ml-exp',
               runs=5, batch_size=34, batch_step=2, seq_step=16, input_length=400)

out_dir = Path(args.out_dir)
out_dir.mkdir(parents=True, exist_ok=True)
runs = args.runs
input_length = args.input_length
batch_size = args.batch_size
batch_step = args.batch_step
seq_step = args.seq_step
exp_type = args.exp_type
exp_name = args.exp_name

In [303]:
res = pd.read_csv(out_dir / f'{exp_name}-res.csv')
energy = pd.read_csv(out_dir / f'{exp_name}-energy.csv',
                     error_bad_lines=False, usecols=[0, 2])
energy = energy[energy['value'].apply(lambda x: is_float(x))]
energy = energy[energy['timestamp'].apply(lambda x: is_float(x))]

energy['value'] = energy['value'].astype(float).div(100)
energy['timestamp'] = energy['timestamp'].astype(float)
infos = []
for model_name in args.models:
    name_s = sanitize(model_name)
    feature_file = out_dir / f'{exp_name}_{name_s}_features.csv'
    feature_values = {k: [] for k in feature_names}
    for bs in list(range(2, batch_size, batch_step)) + [1]:
        for seq_len in range(16, input_length, seq_step):
            
            filename = f'{name_s}_{exp_type}_r{runs}_b{bs}_i{seq_len}.json'
            prof_file = Path(out_dir) / exp_name / filename
            if not prof_file.exists():
                continue
            with open(prof_file) as f:
                prof_info = json.load(f)
            process_record(energy, prof_info, res, feature_values,
                           model_name, bs, runs, seq_len)
    info = pd.DataFrame(data=feature_values)
    infos.append(info)
    info.to_csv(feature_file)
    print(f'{model_name} done.')

<ipython-input-301-8fc70e7b2ad8>:48: RuntimeWarning: invalid value encountered in double_scalars
  rn_std = np.std(res_runs[rn]) / np.mean(res_runs[rn]) * 100


google/mobilebert-uncased done.
bert-base-uncased done.
distilbert-base-uncased done.
roberta-base done.


<ipython-input-301-8fc70e7b2ad8>:45: RuntimeWarning: invalid value encountered in double_scalars
  energy_std = np.std(energy_runs) / energy_mean * 100
<ipython-input-301-8fc70e7b2ad8>:43: RuntimeWarning: invalid value encountered in double_scalars
  gpu_power_std = np.std(gpu_power_runs) / gpu_power_mean * 100


In [304]:
infos[0].sort_values('energy_std')

,batch_size,seq_len,flops,mem_bytes,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std
227,32,224,235.798528,16.000000,5.304041,0.116676,5462108.000000,0.231035,0.205044,0.212802,encoder.layer.0.attention.self.query:Linear,google/mobilebert-uncased,8.446452,10.000000,97.438710,58.632258,0.659991,0.000000,1.532290,1.959924
261,32,384,0.000000,24.093750,2.679315,0.121937,2471122.800000,2.451218,0.399430,0.220465,embeddings.token_type_embeddings:Embedding,google/mobilebert-uncased,8.283250,10.000000,93.583333,42.113333,2.521286,0.000000,3.031040,2.709699
159,24,288,909.508608,49.875000,3.382452,0.007635,3836121.800000,0.632449,0.523798,0.230382,encoder.layer.0.intermediate.dense:Linear,google/mobilebert-uncased,8.384000,10.000000,95.100000,71.340000,2.035165,0.000000,2.732144,2.760890
185,32,32,33.685504,4.000000,16.632993,0.178363,8289168.800000,0.405027,0.062097,0.243909,encoder.layer.0.attention.self.query:Linear,google/mobilebert-uncased,8.502331,10.000000,34.883400,0.061224,0.456933,0.000000,0.375439,200.000000
235,32,256,1074.790400,36.000000,2.671863,0.112153,3300425.000000,2.887320,0.548182,0.259443,encoder.layer.0.attention.self.value:Linear,google/mobilebert-uncased,8.325833,10.000000,93.591667,47.494167,2.103952,0.000000,0.416348,0.440143
60,16,224,470.220800,16.250000,2.844906,0.218521,3479291.400000,2.285784,0.251834,0.334634,encoder.layer.0.attention.self.value:Linear,google/mobilebert-uncased,8.316691,10.000000,93.720588,45.942647,1.825767,0.000000,2.419703,2.376981
143,24,224,176.848896,12.000000,5.429326,0.226352,5450123.000000,1.307924,0.155321,0.427866,encoder.layer.0.attention.self.query:Linear,google/mobilebert-uncased,8.443145,10.000000,96.886694,57.410887,1.540710,0.000000,1.459733,2.161620
164,24,320,252.641280,16.500000,5.304538,0.059338,5407568.600000,0.148933,0.216747,0.433362,encoder.layer.0.attention.self.query:Linear,google/mobilebert-uncased,8.493548,10.000000,97.096774,58.554839,1.365361,0.000000,1.335366,2.197932
178,24,384,303.169536,19.500000,5.234699,0.025978,5452367.200000,1.148425,0.262914,0.462894,encoder.layer.0.attention.self.query:Linear,google/mobilebert-uncased,8.421763,10.000000,97.373763,59.686022,1.341183,0.000000,2.009144,2.670792
3,8,352,92.635136,6.000000,5.567700,1.666853,5233846.400000,1.237347,0.086364,0.510816,encoder.layer.0.attention.self.query:Linear,google/mobilebert-uncased,8.512576,10.000000,89.869508,44.615720,0.929264,0.000000,1.957942,2.870075


In [295]:
infos[1].sort_values('energy_std')

,batch_size,seq_len,flops,mem_bytes,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name
31,32,256,1474943.893504,38259.595703,8.504974,12.000000,99.457615,54.973836,0.472008,0.000000,0.211513,0.260314,36.391895,0.028140,42605233.200000,0.201248,657.031760,0.102774,bert-base-uncased,bert-base-uncased
22,24,224,961420.282368,25230.323975,8.596860,12.000000,99.365992,53.200894,0.473395,0.000000,0.292000,0.377662,23.774385,0.109437,27682954.200000,0.374853,426.648540,0.123985,bert-base-uncased,bert-base-uncased
15,16,256,737471.946752,19131.298828,8.511376,12.000000,99.258716,53.152294,0.594721,0.000000,0.259024,0.258007,18.558849,0.042107,21236622.800000,0.153977,327.726180,0.169065,bert-base-uncased,bert-base-uncased
28,32,160,903239.280640,25371.196533,8.973394,12.000000,99.152756,51.190305,2.867139,0.000000,0.234603,0.534038,21.681742,0.144244,25697474.000000,0.338334,393.524660,0.211776,bert-base-uncased,bert-base-uncased
23,24,256,1106207.920128,28695.447266,8.582893,12.000000,99.358991,54.404517,0.476891,0.000000,0.293524,0.299376,27.543018,0.047771,31906917.800000,0.328585,492.518740,0.238709,bert-base-uncased,bert-base-uncased
14,16,224,640946.854912,16821.185303,8.504461,12.000000,99.177223,52.091489,0.704514,0.000000,0.434345,0.526136,16.070194,0.090546,18454564.000000,0.399453,285.060800,0.318340,bert-base-uncased,bert-base-uncased
20,24,160,677429.460480,19029.077393,8.999789,12.000000,98.825263,51.397895,2.124683,0.000000,0.652216,0.663680,16.159693,0.157443,19113654.400000,0.249358,293.020480,0.345986,bert-base-uncased,bert-base-uncased
18,24,96,400884.576768,13799.830811,8.527719,12.000000,98.392982,48.175439,0.387598,0.000000,0.507412,0.503024,9.753512,0.110010,11080477.800000,0.348315,172.660440,0.360729,bert-base-uncased,bert-base-uncased
21,24,192,818494.129152,22008.200684,8.522534,12.000000,99.298604,51.276770,0.540536,0.000000,0.162186,0.205284,20.227853,0.139202,23288451.400000,0.333542,359.333700,0.363979,bert-base-uncased,bert-base-uncased
25,32,64,353872.482304,15398.797363,8.441176,12.000000,98.027451,45.172549,0.916169,0.000000,0.337229,0.489860,8.701157,0.134634,9712253.800000,0.357379,151.047640,0.369725,bert-base-uncased,bert-base-uncased


In [296]:
infos[2].dropna().sort_values('energy_std')

,batch_size,seq_len,flops,mem_bytes,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name
26,32,96,267218.583552,9009.046875,8.508397,11.000000,97.403333,47.084286,0.716761,0.000000,0.834511,0.939147,6.105381,0.013890,7149497.000000,1.185902,110.030480,0.314705,distilbert-base-uncased,distilbert-base-uncased
14,16,224,320375.881728,7684.554688,8.452039,13.000000,97.748019,47.471304,0.504157,0.000000,1.115519,1.075000,7.692327,0.005014,8812926.600000,0.802637,135.957960,0.416252,distilbert-base-uncased,distilbert-base-uncased
23,24,256,552910.454784,12888.093750,8.606737,13.000000,98.812987,49.563337,0.742626,0.000000,0.779265,0.760980,13.130971,0.009864,15239726.800000,0.306798,233.934260,0.433762,distilbert-base-uncased,distilbert-base-uncased
31,32,256,737213.939712,17184.125000,8.544642,12.000000,99.243914,50.349796,0.378460,0.000000,0.340231,0.349927,17.350395,0.111740,20302028.800000,0.678733,311.653340,0.481888,distilbert-base-uncased,distilbert-base-uncased
29,32,192,545521.139712,13644.093750,9.037067,11.000000,98.573333,49.106667,0.767233,0.000000,0.705705,0.713012,12.735601,0.014315,14981904.600000,0.190907,230.381380,0.494689,distilbert-base-uncased,distilbert-base-uncased
30,32,224,640751.763456,15369.109375,8.886067,13.000000,99.485393,48.773034,2.062584,0.000000,0.157697,0.195151,15.209210,0.008151,17743795.200000,0.433082,273.227680,0.502665,distilbert-base-uncased,distilbert-base-uncased
27,32,128,358754.549760,10464.062500,8.464745,13.000000,98.413095,48.198129,0.378008,0.000000,0.730264,0.692230,8.268359,0.041995,9619756.000000,0.876812,146.896620,0.683801,distilbert-base-uncased,distilbert-base-uncased
11,16,128,179377.274880,5232.031250,8.446462,13.000000,96.554769,44.796923,1.774962,0.000000,1.795926,1.835574,4.342638,0.024293,4821553.200000,1.785010,74.930640,0.792158,distilbert-base-uncased,distilbert-base-uncased
18,24,96,200413.937664,6756.785156,8.520000,13.000000,97.184127,45.711111,1.362019,0.000000,1.173821,1.208262,4.739821,0.017427,5361371.000000,1.757455,82.880320,0.877253,distilbert-base-uncased,distilbert-base-uncased
21,24,192,409140.854784,10233.070312,8.465064,13.000000,98.553842,47.498911,0.558718,0.000000,0.725364,0.773401,9.722713,0.008940,11198226.800000,0.517100,171.861860,0.890473,distilbert-base-uncased,distilbert-base-uncased


In [297]:
infos[3].dropna().sort_values('energy_std')

,batch_size,seq_len,flops,mem_bytes,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name
31,32,256,1474943.909888,38283.125000,8.508525,13.000000,99.541730,55.186016,0.348381,0.000000,0.168512,0.257035,36.452941,0.074229,42826452.600000,0.136187,659.394280,0.176440,roberta-base,roberta-base
14,16,224,640946.862080,16831.150391,8.491075,13.000000,99.160896,52.015162,0.468395,0.000000,0.296474,0.388250,16.102221,0.021939,18519307.400000,0.488594,286.260440,0.239087,roberta-base,roberta-base
30,32,224,1281893.724160,33660.050781,9.029523,13.000000,99.527074,52.851741,1.815107,0.000000,0.279158,0.351909,31.849768,0.063903,37259910.200000,0.296302,574.288680,0.249235,roberta-base,roberta-base
27,32,128,717635.043328,21734.828125,8.517996,13.000000,99.205743,50.748020,0.346976,0.000000,0.220200,0.265863,17.132208,0.144713,19914179.000000,0.490498,307.390720,0.287794,roberta-base,roberta-base
29,32,192,1091325.517824,29360.976562,9.179919,12.000000,99.297950,52.945688,2.702689,0.000000,0.436990,0.448791,26.582488,0.073688,31182663.800000,0.225302,481.171700,0.302256,roberta-base,roberta-base
28,32,160,903239.290880,25385.902344,8.965966,13.000000,99.338792,51.254306,1.464205,0.000000,0.289965,0.406308,21.762289,0.147602,25667877.400000,0.288475,394.860140,0.315285,roberta-base,roberta-base
15,16,256,737471.954944,19142.687500,8.511865,13.000000,99.085371,53.058215,0.267594,0.000000,0.381918,0.403914,18.593617,0.040337,21241280.400000,0.196262,330.010140,0.322031,roberta-base,roberta-base
21,24,192,818494.138368,22021.294922,8.505210,13.000000,99.267227,51.275630,0.252548,0.000000,0.310596,0.301194,20.275331,0.038197,23352421.200000,0.179585,360.348560,0.356916,roberta-base,roberta-base
10,16,96,267256.387584,9205.001953,8.521000,13.000000,98.691667,46.388718,1.069904,0.000000,0.625956,0.619118,6.719930,0.162062,7381716.200000,0.804158,115.028300,0.382395,roberta-base,roberta-base
26,32,96,534512.775168,18407.753906,8.521351,12.000000,98.735135,49.356757,0.862931,0.000000,0.583160,0.595233,12.603253,0.155676,14743108.800000,0.408438,226.760560,0.382748,roberta-base,roberta-base


In [314]:
import toytree

In [315]:
te=toytree.tree("(((root.embeddings.word_embeddings),(root.embeddings.position_embeddings),(root.embeddings.token_type_embeddings),(root.embeddings.LayerNorm),(root.embeddings.dropout),root.embeddings),((((((root.encoder.layer.0.attention.self.query),(root.encoder.layer.0.attention.self.key),(root.encoder.layer.0.attention.self.value),(root.encoder.layer.0.attention.self.dropout),root.encoder.layer.0.attention.self),((root.encoder.layer.0.attention.output.dense),(root.encoder.layer.0.attention.output.LayerNorm),(root.encoder.layer.0.attention.output.dropout),root.encoder.layer.0.attention.output),root.encoder.layer.0.attention),((root.encoder.layer.0.intermediate.dense),root.encoder.layer.0.intermediate),((root.encoder.layer.0.output.dense),(root.encoder.layer.0.output.LayerNorm),(root.encoder.layer.0.output.dropout),root.encoder.layer.0.output),root.encoder.layer.0),((((root.encoder.layer.1.attention.self.query),(root.encoder.layer.1.attention.self.key),(root.encoder.layer.1.attention.self.value),(root.encoder.layer.1.attention.self.dropout),root.encoder.layer.1.attention.self),((root.encoder.layer.1.attention.output.dense),(root.encoder.layer.1.attention.output.LayerNorm),(root.encoder.layer.1.attention.output.dropout),root.encoder.layer.1.attention.output),root.encoder.layer.1.attention),((root.encoder.layer.1.intermediate.dense),root.encoder.layer.1.intermediate),((root.encoder.layer.1.output.dense),(root.encoder.layer.1.output.LayerNorm),(root.encoder.layer.1.output.dropout),root.encoder.layer.1.output),root.encoder.layer.1),root.encoder.layer),root.encoder),((root.pooler.dense),(root.pooler.activation),root.pooler),root);")

In [320]:
te.draw(height=1200, node_hover=True, node_sizes=12, tip_labels_align=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="485.0px" height="1200.0px" viewBox="0 0 485.0 1200.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taeeef92b6d0642ab8c310bd1acb5e293"> idx: 0
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.1.attention.self idx: 1
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.1.attention.self.dropout idx: 2
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.1.attention.self.value idx: 3
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.1.attention.self.key idx: 4
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.1.attention.self.query idx: 5
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.1.attention.output idx: 6
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.1.attention.output.dropout idx: 7
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.1.attention.output.LayerNorm idx: 8
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.1.attention.output.dense idx: 9
dist: 1.0000
support: 0.0000
height: 2.0000
name: root.encoder.layer.1.attention idx: 10
dist: 1.0000
support: 0.0000
height: 2.0000
name: root.encoder.layer.1.output idx: 11
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.1.output.dropout idx: 12
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.1.output.LayerNorm idx: 13
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.1.output.dense idx: 14
dist: 1.0000
support: 0.0000
height: 2.0000
name: root.encoder.layer.1.intermediate idx: 15
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.1.intermediate.dense idx: 16
dist: 1.0000
support: 0.0000
height: 3.0000
name: root.encoder.layer.1 idx: 17
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.0.attention.self idx: 18
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.0.attention.self.dropout idx: 19
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.0.attention.self.value idx: 20
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.0.attention.self.key idx: 21
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.0.attention.self.query idx: 22
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.0.attention.output idx: 23
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.0.attention.output.dropout idx: 24
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.0.attention.output.LayerNorm idx: 25
dist: 1.0000
support: 0.0000
height: 0.0000
name: root.encoder.layer.0.attention.output.dense idx: 26
dist: 1.0000
support: 0.0000
height: 2.0000
name: root.encoder.layer.0.attention idx: 27
dist: 1.0000
support: 0.0000
height: 2.0000
name: root.encoder.layer.0.output idx: 28
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.0.output.dropout idx: 29
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.0.output.LayerNorm idx: 30
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.0.output.dense idx: 31
dist: 1.0000
support: 0.0000
height: 2.0000
name: root.encoder.layer.0.intermediate idx: 32
dist: 1.0000
support: 0.0000
height: 1.0000
name: root.encoder.layer.0.intermediate.dense idx: 33
dist: 1.0000
support: 0.0000
height: 3.0000
name: root.encoder.layer.0 idx: 34
dist: 1.0000
support: 0.0000
height: 4.0000
name: root.encoder.layer idx: 35
dist: 1.0000
support: 0.0000
height: 5.0000
name: root.encoder idx: 36
dist: 1.0000
support: 0.0000
height: 5.0000
name: